In [ ]:
# |default_exp mcp_weather_server

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from typing import Any
import asyncio
import nest_asyncio
import httpx
#from fastmcp import FastMCP
from mcp.server.fastmcp import FastMCP

In [ ]:
#| export
# Initialize FastMCP server
mcp = FastMCP("weather")

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"



In [ ]:
#| export
async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {"User-Agent": USER_AGENT, "Accept": "application/geo+json"}
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None



In [ ]:
# Test that make_nws_request actually makes HTTP requests

# Test with a simple endpoint that echoes back request info
test_url = "https://httpbin.org/json"
result = await make_nws_request(test_url)
assert result is not None, "Request should return a dict, not None"
assert isinstance(result, dict), "Result should be a dictionary"
assert "slideshow" in result or len(result) > 0, "Should receive valid JSON response"

# Test error handling - invalid URL should return None
invalid_url = "https://invalid-domain-that-does-not-exist-12345.com/api"
result = await make_nws_request(invalid_url)
assert result is None, "Invalid URL should return None"


[01/07/26 14:25:03] INFO     HTTP Request: GET https://httpbin.org/json "HTTP/1.1 200 OK"           ]8;id=627142;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=435969;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\

In [ ]:
#| export
def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    return f"""
Event: {props.get("event", "Unknown")}
Area: {props.get("areaDesc", "Unknown")}
Severity: {props.get("severity", "Unknown")}
Description: {props.get("description", "No description available")}
Instructions: {props.get("instruction", "No specific instructions provided")}
"""



In [ ]:
# Test format_alert function
test_feature = {
    "properties": {
        "event": "Tornado Warning",
        "areaDesc": "Los Angeles County, CA",
        "severity": "Extreme",
        "description": "A tornado has been spotted in the area.",
        "instruction": "Seek shelter immediately."
    }
}

result = format_alert(test_feature)
assert "Tornado Warning" in result
assert "Los Angeles County, CA" in result
assert "Extreme" in result
assert "A tornado has been spotted" in result
assert "Seek shelter immediately" in result

# Test format_alert with missing fields
test_feature_minimal = {
    "properties": {
        "event": "Flood Watch"
    }
}

result = format_alert(test_feature_minimal)
assert "Flood Watch" in result
assert "Unknown" in result  # Should have Unknown for missing fields
assert "No description available" in result
assert "No specific instructions provided" in result



In [ ]:
#| export
@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)



In [ ]:
# Test that get_alerts makes real API requests
result = await get_alerts("CA")
assert isinstance(result, str), "get_alerts should return a string"
assert len(result) > 0, "Result should not be empty"
# Should either have alerts or the "no alerts" message
assert ("No active alerts" in result or "Event:" in result or "Unable to fetch" in result), \
    "Result should indicate alerts status"


# Test get_alerts with another state (NY - New York)
result = await get_alerts("NY")
assert isinstance(result, str), "get_alerts should return a string"
assert len(result) > 0, "Result should not be empty"

[01/07/26 14:25:04] INFO     HTTP Request: GET https://api.weather.gov/alerts/active/area/CA        ]8;id=114784;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=633916;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[01/07/26 14:25:05] INFO     HTTP Request: GET https://api.weather.gov/alerts/active/area/NY        ]8;id=987730;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=136958;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [ ]:
# Test get_alerts result format when alerts exist
# If there are alerts, they should be formatted with format_alert and separated by "---"
result = await get_alerts("CA")
if "Event:" in result:
    # If alerts exist, verify formatting
    assert "---" in result or result.count("Event:") >= 1, \
        "Multiple alerts should be separated by '---'"
    assert "Event:" in result, "Alerts should contain formatted event information"


                    INFO     HTTP Request: GET https://api.weather.gov/alerts/active/area/CA        ]8;id=752235;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=502253;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [ ]:
#| export
@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
{period["name"]}:
Temperature: {period["temperature"]}°{period["temperatureUnit"]}
Wind: {period["windSpeed"]} {period["windDirection"]}
Forecast: {period["detailedForecast"]}
"""
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)



In [ ]:
# Test get_forecast result format when forecast exists
# with real coordinates (San Francisco, CA)
# Latitude: 37.7749, Longitude: -122.4194

result = await get_forecast(37.7749, -122.4194)
if "Temperature:" in result:
    # Verify forecast formatting
    assert ":" in result, "Forecast should have formatted periods"
    assert "°" in result or "Temperature:" in result, "Forecast should include temperature"
    # Check that periods are separated by "---"
    assert "---" in result or result.count("Temperature:") <= 5, \
        "Forecast should show at most 5 periods separated by '---'"


                    INFO     HTTP Request: GET https://api.weather.gov/points/37.7749,-122.4194     ]8;id=86278;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=643903;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: GET                                                      ]8;id=622734;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=945525;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://api.weather.gov/gridpoints/MTR/85,105/forecast "HTTP/1.1 200                  
                             OK"                                                                                   

In [ ]:
# Test get_forecast with coordinates that might not have forecast data
# Using coordinates in the middle of the ocean (should handle gracefully)
result = await get_forecast(0.0, 0.0)
assert isinstance(result, str), "get_forecast should always return a string"
assert len(result) > 0, "Result should not be empty"
# Should return an error message if location is invalid
assert "Unable to fetch" in result, \
    "Should return error message or forecast data"


                    INFO     HTTP Request: GET https://api.weather.gov/points/0.0,0.0 "HTTP/1.1 404 ]8;id=185427;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=514566;file:///Users/olgasoldatenko/work/newer/stringdale/.venv/lib/python3.11/site-packages/httpx/_client.py#1740\1740]8;;\
                             Not Found"                                                                            

In [ ]:
#| export
def main():
    # Initialize and run the server
    mcp.run(transport="stdio")

In [ ]:
#| export
if __name__ == "__main__":
    main()

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()